## Imports

In [67]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import os

In [68]:
def grouped_apply_agg(df, group, cols, names, functions):

    grouped = df.sort_values('date').groupby(group, sort=False)    

    for col in cols:
        for name, function in zip(names, functions):
            df[name + "_" + col] = grouped[col].apply(function).reset_index(level=[0,1,2], drop=True)

    return df


## Importing data

In [69]:
data_dir = "data"

info_db = pd.read_csv(os.path.join(data_dir, "train.csv"), index_col=0)
store_db = pd.read_csv(os.path.join(data_dir, "stores.csv"), index_col=0)
oil_db = pd.read_csv(os.path.join(data_dir, "oil.csv"))

holidays_db = pd.read_csv(os.path.join(data_dir, "holidays_events.csv"))

local_holidays_db = holidays_db[holidays_db['locale'] == 'Local']
local_holidays_db = holidays_db.rename(columns={'locale_name': 'city', 'type': 'h_type_loc', 'description': 'h_description_loc', 'transferred': 'h_transferred_loc'})
local_holidays_db = local_holidays_db.drop(columns=['locale'])

national_holidays_db = holidays_db[holidays_db['locale'] == 'National']
national_holidays_db = national_holidays_db.rename(columns={'type': 'h_type_nat', 'description': 'h_description_nat', 'transferred': 'h_transferred_nat'})
national_holidays_db = national_holidays_db.drop(columns=['locale', 'locale_name'])

transactions_db = pd.read_csv(os.path.join(data_dir, "transactions.csv"))

db = info_db.merge(store_db, on="store_nbr", how="left")

db = db.merge(oil_db, on="date", how="left")
db['dcoilwtico'] = db['dcoilwtico'].fillna(method='bfill')


db = db.merge(national_holidays_db, on="date", how="left")
db['h_type_nat'] = db['h_type_nat'].fillna('no_holiday')
db['h_description_nat'] = db['h_description_nat'].fillna('no_holiday')
db['h_transferred_nat'] = db['h_transferred_nat'].fillna('no_holiday')

db = db.merge(local_holidays_db, on=["date", 'city'], how="left")
db['h_type_loc'] = db['h_type_loc'].fillna('no_holiday')
db['h_description_loc'] = db['h_description_loc'].fillna('no_holiday')
db['h_transferred_loc'] = db['h_transferred_loc'].fillna('no_holiday')


db = db.merge(transactions_db, on=["date", "store_nbr"], how="left")
db['transactions'] = db['transactions'].fillna(0)

db['date'] = pd.to_datetime(db['date'])

db['year'] = pd.to_datetime(db['date']).dt.year
db['month'] = pd.to_datetime(db['date']).dt.month
db['day'] = pd.to_datetime(db['date']).dt.day
db['day_of_week'] = db['date'].dt.dayofweek

db = db[db['date'] >= '2017-02-01']

group = ['family', 'day_of_week', 'store_nbr']
cols = ['sales', 'transactions']
names = ['dow_avg', 'dow_rolling_3', 'dow_rolling_7']
agg_funcs = [
    lambda x: x.expanding().mean().shift(1).fillna(0),
    lambda x: x.rolling(window=3).mean().shift(1).fillna(0),
    lambda x: x.rolling(window=7).mean().shift(1).fillna(0),
]
dow_aves_db = grouped_apply_agg(
    db[['date', 'store_nbr', 'transactions', 'family', 'sales', 'day_of_week']].copy(),
    group,
    cols, 
    names, 
    agg_funcs
).drop(columns=['sales', 'transactions', 'day_of_week'])

db = db.merge(dow_aves_db, on=['date','store_nbr', 'family'], how='left')

group = ['family', 'store_nbr']
cols = ['sales', 'transactions']
names = ['rolling_7', 'rolling_14']
agg_funcs = [
    lambda x: x.rolling(window=7).mean().shift(1).fillna(0),
    lambda x: x.rolling(window=14).mean().shift(1).fillna(0),
]
rolling_db = grouped_apply_agg(
    db[['date', 'store_nbr', 'transactions', 'family', 'sales']].copy(),
    group,
    cols, 
    names, 
    agg_funcs
).drop(columns=['sales', 'transactions'])

db = db.merge(rolling_db, on=['date','store_nbr', 'family'], how='left')

db = db.sort_index()
db = db.drop(columns='transactions')


db = db.drop(columns=["date"])

In [70]:
display(db.head(20))
display(db.info())

,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,h_type_nat,...,dow_avg_sales,dow_rolling_3_sales,dow_rolling_7_sales,dow_avg_transactions,dow_rolling_3_transactions,dow_rolling_7_transactions,rolling_7_sales,rolling_14_sales,rolling_7_transactions,rolling_14_transactions
0,1,AUTOMOTIVE,7.000,0,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,1,BEAUTY,7.000,1,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,1,BEVERAGES,2399.000,28,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,1,BOOKS,1.000,0,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
5,1,BREAD/BAKERY,512.612,10,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
6,1,CELEBRATION,3.000,0,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
7,1,CLEANING,1070.000,18,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,3.428571,0.000000,1592.142857,0.000000
8,1,DAIRY,937.000,17,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,3.285714,0.000000,1593.857143,0.000000
9,1,DELI,156.980,5,Quito,Pichincha,D,13,53.9,no_holiday,...,0.0,0.0,0.0,0.0,0.0,0.0,3.571429,0.000000,1593.428571,0.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349272 entries, 0 to 349271
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   store_nbr                   349272 non-null  int64  
 1   family                      349272 non-null  object 
 2   sales                       349272 non-null  float64
 3   onpromotion                 349272 non-null  int64  
 4   city                        349272 non-null  object 
 5   state                       349272 non-null  object 
 6   type                        349272 non-null  object 
 7   cluster                     349272 non-null  int64  
 8   dcoilwtico                  349272 non-null  float64
 9   h_type_nat                  349272 non-null  object 
 10  h_description_nat           349272 non-null  object 
 11  h_transferred_nat           349272 non-null  object 
 12  h_type_loc                  349272 non-null  object 
 13  h_description_

None

## Renaming and Assigning classes

In [71]:
col_rename = {
    'dcoilwtico': 'oil',
    'type': 'store_type',
}

db = db.rename(columns=col_rename)

col_str_to_class_label = {}

for col in db.columns:
    # skip if the column is numeric
    if is_numeric_dtype(db[col]):
        continue
    str_to_class = {}
    for i, val in enumerate(db[col].unique()):
        str_to_class[val] = i
    col_str_to_class_label[col] = str_to_class

for col, dict in col_str_to_class_label.items():
    db[col] = db[col].map(dict)

display(db.head())

,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,oil,h_type_nat,...,dow_avg_sales,dow_rolling_3_sales,dow_rolling_7_sales,dow_avg_transactions,dow_rolling_3_transactions,dow_rolling_7_transactions,rolling_7_sales,rolling_14_sales,rolling_7_transactions,rolling_14_transactions
0,1,0,7.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,7.0,1,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,2399.0,28,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,1.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculating Correlation

In [72]:
rows = [
    'sales',
    'store_nbr',
    'family',
    'city',
    'state',
    'store_type',
    'cluster',
    'oil',
    'month',
    'day',
    'h_type_nat',
    'h_type_loc',
    'dow_avg_sales',
    'dow_rolling_3_sales',
    'rolling_7_sales',
    'rolling_14_sales',
    'dow_avg_transactions',
    'dow_rolling_3_transactions',
    'rolling_7_transactions',
    'rolling_14_transactions'
]

cov_db = db[rows].copy()
for col in cov_db.columns:
    cov_db[col] = cov_db[col].astype(np.float32)

cov_matrix = cov_db.corr()

display(cov_matrix)

del cov_db

,sales,store_nbr,family,city,state,store_type,cluster,oil,month,day,h_type_nat,h_type_loc,dow_avg_sales,dow_rolling_3_sales,rolling_7_sales,rolling_14_sales,dow_avg_transactions,dow_rolling_3_transactions,rolling_7_transactions,rolling_14_transactions
sales,1.000000,6.043635e-02,-1.120805e-01,-8.447596e-02,-8.702399e-02,1.177458e-01,3.808259e-02,-1.368423e-04,1.540230e-03,-1.481546e-02,4.216835e-03,-5.140129e-03,0.950667,0.914263,0.001841,0.002336,2.007082e-01,1.812437e-01,-0.004944,-0.004802
store_nbr,0.060436,1.000000e+00,1.845645e-17,5.886759e-01,5.874486e-01,5.719704e-01,-5.928399e-02,-1.942036e-15,4.025876e-15,9.587841e-17,1.182884e-15,1.454748e-02,0.052615,0.056975,0.025343,0.024229,1.599111e-01,1.634294e-01,-0.002745,-0.002783
family,-0.112080,1.845645e-17,1.000000e+00,4.317603e-18,8.535002e-18,1.524324e-17,1.156532e-18,4.679586e-16,-1.032069e-15,1.729931e-15,-5.002488e-16,3.845064e-16,-0.109831,-0.106639,-0.002299,-0.000380,5.746073e-16,9.169662e-16,-0.001208,0.000011
city,-0.084476,5.886759e-01,4.317603e-18,1.000000e+00,9.852099e-01,-1.113357e-01,-4.230199e-01,-5.725284e-15,5.457726e-15,7.762781e-16,1.669744e-17,2.369479e-02,-0.095839,-0.081105,0.005352,0.004170,-4.441746e-01,-3.621559e-01,-0.005859,-0.006035
state,-0.087024,5.874486e-01,8.535002e-18,9.852099e-01,1.000000e+00,-1.073820e-01,-4.167306e-01,-1.022522e-14,1.392214e-14,4.816646e-16,6.271591e-16,3.020977e-02,-0.098613,-0.083555,0.004244,0.002905,-4.524061e-01,-3.690270e-01,-0.004771,-0.004996
store_type,0.117746,5.719704e-01,1.524324e-17,-1.113357e-01,-1.073820e-01,1.000000e+00,3.880565e-01,5.493436e-15,-5.040546e-15,8.499653e-17,1.123667e-15,2.421631e-03,0.111557,0.111330,0.015301,0.014784,4.356192e-01,4.064740e-01,0.004248,0.004072
cluster,0.038083,-5.928399e-02,1.156532e-18,-4.230199e-01,-4.167306e-01,3.880565e-01,1.000000e+00,-9.308509e-15,9.176071e-15,-5.516863e-16,-1.044246e-16,-1.556458e-02,0.037685,0.036476,-0.000948,-0.001052,1.742425e-01,1.630999e-01,0.001724,0.002375
oil,-0.000137,-1.942036e-15,4.679586e-16,-5.725284e-15,-1.022522e-14,5.493436e-15,-9.308509e-15,1.000000e+00,-6.534500e-01,-7.347700e-02,5.714987e-02,-2.194533e-02,-0.018824,-0.060956,0.083356,0.081785,-8.606414e-02,-2.604783e-01,0.287713,0.268765
month,0.001540,4.025876e-15,-1.032069e-15,5.457726e-15,1.392214e-14,-5.040546e-15,9.176071e-15,-6.534500e-01,1.000000e+00,-8.508738e-02,6.409378e-02,4.573976e-02,0.020550,0.061895,-0.086221,-0.084702,9.475794e-02,2.725798e-01,-0.312910,-0.292561
day,-0.014815,9.587841e-17,1.729931e-15,7.762781e-16,4.816646e-16,8.499653e-17,-5.516863e-16,-7.347700e-02,-8.508738e-02,1.000000e+00,4.045355e-02,3.430264e-02,0.015846,0.020214,0.034772,0.034208,7.658432e-02,8.334263e-02,0.131190,0.122941


## Saving Data

In [73]:
db.to_csv(os.path.join(data_dir, "train_data.csv"), index=False)

## Making Test Data Set

In [11]:
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"), index_col=0)
index_col = test_df.index

test_df = test_df.merge(store_db, on="store_nbr", how="left")
test_df = test_df.merge(oil_db, on="date", how="left")
test_df['dcoilwtico'] = test_df['dcoilwtico'].fillna(method='bfill')

test_df = test_df.merge(holidays_db, on="date", how="left")
test_df['h_type'] = test_df['h_type'].fillna('no_holiday')
test_df['h_locale'] = test_df['h_locale'].fillna('no_locale')
test_df['h_locale_name'] = test_df['h_locale_name'].fillna('no_locale_name')

test_df['date'] = pd.to_datetime(test_df['date'])

test_df['year'] = pd.to_datetime(test_df['date']).dt.year
test_df['month'] = pd.to_datetime(test_df['date']).dt.month
test_df['day'] = pd.to_datetime(test_df['date']).dt.day
test_df['day_of_week'] = test_df['date'].dt.dayofweek

test_df = test_df.rename(columns=col_rename)

display(test_df.head())

for col, dict in col_str_to_class_label.items():
    print(col, dict)
    test_df[col] = test_df[col].map(dict)

most_recent_dow_aves = db.groupby(['store_nbr', 'family', 'day_of_week']).tail(1)
most_recent_dow_aves = most_recent_dow_aves[['store_nbr', 'family', 'day_of_week', 'dow_avg_sales', 'rolling_7_sales', 'rolling_14_sales', 'rolling_30_sales', 'dow_avg_transactions', 'rolling_7_transactions', 'rolling_14_transactions', 'rolling_30_transactions']]

test_df = test_df.merge(most_recent_dow_aves, on=['store_nbr', 'family', 'day_of_week'], how='left')
test_df = test_df.drop(columns=["date"])
test_df.index = index_col

display(test_df.head())



,date,store_nbr,family,onpromotion,city,state,store_type,cluster,oil,h_type,h_locale,h_locale_name,transferred,holiday_count,year,month,day,day_of_week
0,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8,no_holiday,no_locale,no_locale_name,NaN,NaN,2017,8,16,2
1,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8,no_holiday,no_locale,no_locale_name,NaN,NaN,2017,8,16,2
2,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8,no_holiday,no_locale,no_locale_name,NaN,NaN,2017,8,16,2
3,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8,no_holiday,no_locale,no_locale_name,NaN,NaN,2017,8,16,2
4,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8,no_holiday,no_locale,no_locale_name,NaN,NaN,2017,8,16,2


family {'AUTOMOTIVE': 0, 'BABY CARE': 1, 'BEAUTY': 2, 'BEVERAGES': 3, 'BOOKS': 4, 'BREAD/BAKERY': 5, 'CELEBRATION': 6, 'CLEANING': 7, 'DAIRY': 8, 'DELI': 9, 'EGGS': 10, 'FROZEN FOODS': 11, 'GROCERY I': 12, 'GROCERY II': 13, 'HARDWARE': 14, 'HOME AND KITCHEN I': 15, 'HOME AND KITCHEN II': 16, 'HOME APPLIANCES': 17, 'HOME CARE': 18, 'LADIESWEAR': 19, 'LAWN AND GARDEN': 20, 'LINGERIE': 21, 'LIQUOR,WINE,BEER': 22, 'MAGAZINES': 23, 'MEATS': 24, 'PERSONAL CARE': 25, 'PET SUPPLIES': 26, 'PLAYERS AND ELECTRONICS': 27, 'POULTRY': 28, 'PREPARED FOODS': 29, 'PRODUCE': 30, 'SCHOOL AND OFFICE SUPPLIES': 31, 'SEAFOOD': 32}
city {'Quito': 0, 'Cayambe': 1, 'Latacunga': 2, 'Riobamba': 3, 'Ibarra': 4, 'Santo Domingo': 5, 'Guaranda': 6, 'Puyo': 7, 'Ambato': 8, 'Guayaquil': 9, 'Salinas': 10, 'Daule': 11, 'Babahoyo': 12, 'Quevedo': 13, 'Playas': 14, 'Libertad': 15, 'Cuenca': 16, 'Loja': 17, 'Machala': 18, 'Esmeraldas': 19, 'Manta': 20, 'El Carmen': 21}
state {'Pichincha': 0, 'Cotopaxi': 1, 'Chimborazo': 2,

,store_nbr,family,onpromotion,city,state,store_type,cluster,oil,h_type,h_locale,...,day,day_of_week,dow_avg_sales,rolling_7_sales,rolling_14_sales,rolling_30_sales,dow_avg_transactions,rolling_7_transactions,rolling_14_transactions,rolling_30_transactions
id,,,,,,,,,,,,,,,,,,,,,
3000888,1,0,0,0,0,0,13,46.8,1,1,...,16,2,3.531381,3.428571,3.928571,4.133333,1863.393305,1888.857143,1863.857143,1859.3
3000889,1,1,0,0,0,0,13,46.8,1,1,...,16,2,0.000000,0.000000,0.000000,0.000000,1863.393305,1888.857143,1863.857143,1859.3
3000890,1,2,2,0,0,0,13,46.8,1,1,...,16,2,2.615063,3.857143,4.428571,3.700000,1863.393305,1888.857143,1863.857143,1859.3
3000891,1,3,20,0,0,0,13,46.8,1,1,...,16,2,1845.485356,2456.142857,2471.571429,2504.466667,1863.393305,1888.857143,1863.857143,1859.3
3000892,1,4,0,0,0,0,13,46.8,1,1,...,16,2,0.167364,0.000000,0.571429,0.733333,1863.393305,1888.857143,1863.857143,1859.3


In [13]:
test_df.to_csv(os.path.join(data_dir, "test_data.csv"), index=True)